In [3]:
from pymodbus.client.sync import ModbusTcpClient

client = ModbusTcpClient('192.168.0.52')
client.write_coil(1, True)
result = client.read_coils(1,1)
print(result.bits[0])
print()
client.close()

ImportError: No module named 'pymodbus'

In [2]:
'''
192.168.0.52
port:
502 id 2 start address 40001-40125 word
503 id 3
...
507 id 7
'''

client = ModbusTcpClient('192.168.0.52', port=502)
print(client.read_holding_registers(address=1, count=124, unit=1).registers)
client.close()

NameError: name 'ModbusTcpClient' is not defined

## Synchronized
no print    
CPU times: user 4 ms, sys: 0 ns, total: 4 ms  
Wall time: 2.31 s

In [74]:
%%time
from pymodbus.client.sync import ModbusTcpClient

if __name__ == "__main__":
    for port in range(502, 508):
        client = ModbusTcpClient('192.168.0.52', port=port)
        reg = client.read_holding_registers(address=0, count=124, unit=1).registers
        #print('port:{} | registers:{}'.format(port, reg))
        client.close()
    print("Done!")

Done!
CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 2.37 s


## Multiprocess
no print  
CPU times: user 8 ms, sys: 24 ms, total: 32 ms  
Wall time: 341 ms


In [49]:
%%time
from pymodbus.client.sync import ModbusTcpClient
import multiprocessing

def rhr_cycle(ip, port, address=0, count=124, unit=1):
    client = ModbusTcpClient(ip, port=port)
    reg = client.read_holding_registers(address=address, count=count, unit=unit).registers
    #print('port:{} | registers:{}'.format(port, reg))
    client.close()

if __name__ == "__main__":
    
    ip = '192.168.0.52'
    ports = range(502,508)
    processes = []
    for port in ports:
        read_process = multiprocessing.Process(target=rhr_cycle, args=(ip,port))
        processes.append(read_process)
    for process in processes:
        process.start()
        
    for process in processes:
        process.join()


    print("Done!")

Done!
CPU times: user 8 ms, sys: 24 ms, total: 32 ms
Wall time: 341 ms


# gevent async
no print  
CPU times: user 4 ms, sys: 4 ms, total: 8 ms  
Wall time: 2.34 s  

In [68]:
%%time
import gevent
import os

def rhr_cycle(ip, port, address=0, count=124, unit=1):
    client = ModbusTcpClient(ip, port=port)
    reg = client.read_holding_registers(address=address, count=count, unit=unit).registers
    #print('port:{} | registers:{}'.format(port, reg))
    client.close()
    
if __name__ == "__main__":
    ip = '192.168.0.52'
    ports = range(502,508)
    threads = [gevent.spawn(rhr_cycle, ip, port) for port in ports]
    gevent.joinall(threads)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 2.32 s


## async with multiprocess
CPU times: user 4 ms, sys: 12 ms, total: 16 ms  
Wall time: 817 ms

In [83]:
%%time
from pymodbus.client.sync import ModbusTcpClient
from multiprocessing import Pool, TimeoutError
import os

def rhr_cycle(ip, port, address=0, count=124, unit=1):
    client = ModbusTcpClient(ip, port=port)
    reg = client.read_holding_registers(address=address, count=count, unit=unit).registers
    ret_string = 'ppid:{} | pid:{} | port:{} | register:{}'.format(os.getppid(), os.getpid(), port, reg)
    client.close()
    #print(ret_string)

def async_task(task, ip, ports, intervals=1):
    threads = [gevent.spawn(task, ip, port) for port in ports]
    gevent.joinall(threads)

if __name__ == "__main__":
    
    ip = '192.168.0.52'
    results = []
    num_process = 3
    ports_assign = {
        0: [502,503],
        1: [504,505],
        2: [506,507]
    }
    with Pool(processes=num_process) as pool:
        results = [pool.apply_async(async_task, args=(rhr_cycle, ip, ports_assign[ports])) for ports in ports_assign]
        for result in results:
            result.get()
    print("Done!")

Done!
CPU times: user 4 ms, sys: 20 ms, total: 24 ms
Wall time: 822 ms


## multiprocess duration test

In [232]:
a = ModbusTcpClient('10.101.100.123', port=503)
a.port
print(math.floor(time.time()))
print(time.time())

1510561463
1510561463.274439


In [6]:
import sys
import os
import time
import math
from pymodbus.client.sync import ModbusTcpClient
from pymodbus.exceptions import ConnectionException
from multiprocessing import Pool, TimeoutError

start_time = time.time()
ip = '192.168.0.52'
ports = range(502,508)
clients = list([ModbusTcpClient(ip, port=port) for port in ports])

def runner(client):
    global start_time
    next_time = math.floor(time.time())
    while True:
        if (time.time() - start_time > 60*60*10):
            break
        if time.time() > next_time:
            next_time += 1
        else:
            sleep_time = max(0.2, next_time - time.time())
            time.sleep(sleep_time)
            continue
        
        
        reg = client.read_holding_registers(address=0, count=124, unit=1).registers
        #print(reg)
        info = 'time elap:{:1.2f} | port:{} | pid:{}'.format
        print(info(time.time()-start_time, client.port, os.getpid()))



if __name__ == "__main__":
    pool = Pool(6)

    while True:
        try:
            results = pool.map_async(runner, clients)
            results.get()
        except ConnectionException:
            print('Connection Exception')
            pool.terminate()
            continue
        except KeyboardInterrupt:
            pool.terminate()
            break
        # except: # other errors
        #     e = sys.exc_info()[0]
        #     print('Other error: {}'.format(e))
        #     pool.terminate()

    pool.close()
    pool.join()

time elap:0.38 | port:503 | pid:12724
time elap:0.38 | port:502 | pid:12723
time elap:0.39 | port:505 | pid:12726
time elap:0.38 | port:504 | pid:12725
time elap:0.39 | port:506 | pid:12727
time elap:0.39 | port:507 | pid:12728
time elap:0.88 | port:502 | pid:12723
time elap:0.88 | port:504 | pid:12725
time elap:0.88 | port:503 | pid:12724
time elap:0.89 | port:505 | pid:12726
time elap:0.89 | port:507 | pid:12728
time elap:0.89 | port:506 | pid:12727
time elap:1.78 | port:502 | pid:12723
time elap:1.78 | port:503 | pid:12724
time elap:1.78 | port:504 | pid:12725
time elap:1.79 | port:506 | pid:12727
time elap:1.79 | port:507 | pid:12728
time elap:1.79 | port:505 | pid:12726


Process ForkPoolWorker-34:
Process ForkPoolWorker-32:
Process ForkPoolWorker-35:
Process ForkPoolWorker-33:
Process ForkPoolWorker-31:
Traceback (most recent call last):
Process ForkPoolWorker-36:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/eric/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/eric/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/eric/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/eric/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/eric/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/eric/anaconda3/lib/python3.6/multiprocessing/process.py", l

In [214]:
print(time.time())
print(round(time.time()))

1510560999.4652064
1510560999


In [102]:
%%time
# set interval to 1 request per second
# record data
# record success/fail
# ConnectionException

from pymodbus.client.sync import ModbusTcpClient
from multiprocessing import Pool, TimeoutError
from time import sleep
import os

def rhr_cycle(ip, port, address=0, count=124, unit=1):
    client = ModbusTcpClient(ip, port=port)
    reg = client.read_holding_registers(address=address, count=count, unit=unit).registers
    ret_string = 'ppid:{} | pid:{} | port:{} | register:{}'.format(os.getppid(), os.getpid(), port, reg)
    client.close()
    print(ret_string)
    return ret_string

if __name__ == "__main__":
    
    ip = '192.168.0.52'
    ports = range(502,508)
    timer = 5 # in seconds
    
    with Pool(processes=6) as pool:
        results = [pool.apply_async(rhr_cycle, args=(ip,port)) for port in ports]
        
        #sleep(3)
        for result in results:
            print(result[0])
    print("Done!")

TypeError: 'ApplyResult' object does not support indexing

# Further testing

In [111]:
%%time
# CPU times: user 8 ms, sys: 16 ms, total: 24 ms
# Wall time: 22.3 ms

from pymodbus.client.sync import ModbusTcpClient
from multiprocessing import Pool, TimeoutError
import os 

def rhr_cycle(ip, port, address=0, count=124, unit=1):
    client = ModbusTcpClient(ip, port=port)
    reg = client.read_holding_registers(address=address, count=count, unit=unit).registers
    ret_string = 'ppid:{} | pid:{} | port:{} | register:{}'.format(os.getppid(), os.getpid(), port, reg)
    client.close()
    return ret_string

if __name__ == "__main__":
    
    ip = '192.168.0.52'
    ports = range(502,508)
    results = []
    with Pool(processes=8) as pool:
        results = [pool.apply_async(rhr_cycle, args=(ip,port)) for port in ports]
        
        for result in results:
            print(result.get())
    print("Done!")

ConnectionException: Modbus Error: [Connection] Failed to connect[192.168.0.52:502]